# Final Version

# 0. Imports

In [20]:
# Disable Warnings
import warnings
warnings.filterwarnings("ignore")

In [21]:
# Importing Dependencies

# OpenCV
import cv2

# Mediapipe Blazepose
import mediapipe as mp

# Path Management
import os

# General Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [22]:
# Mediapose Utilities

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

# 0.1 Creating Visualizations with Angles

In [23]:
# calculating the joint angles
def calculate_angle(a, b, c):
    # we will be calculating angle ABC
    
    # extract unit vector BA:
    v1 = a - b
    v1_u = v1/np.linalg.norm(v1)
    # extract unit vector BC:
    v2 = c - b
    v2_u = v2/np.linalg.norm(v2)

    return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))



def create_visualization(video_path):
    ## these are 16 tuples of keypoints to extract 18 relevant angles
    keypoint_index = np.array([
        [33, 16, 14],
        [16, 14, 12],
        [14, 12, 11],
        [14, 12, 24],
        [ 0, 34, 12],
        [ 0, 34, 36],
        [12, 11, 13],
        [13, 11, 23],
        [11, 13, 15],
        [13, 15, 35],
        [12, 24, 26],
        [26, 24, 23],
        [24, 23, 25],
        [11, 23, 25],
        [24, 26, 28],
        [23, 25, 27],
        [26, 28, 32],
        [25, 27, 31]
    ])

    # no. of angles to consider: k
    k = keypoint_index.shape[0]

    # we make a boolean array to store the angles we would like to visualize
    angles_to_visualize = np.zeros(k)

    # set the angles to be visualized to 1 (True)
    angles_to_visualize[1] = 1
    angles_to_visualize[3] = 1
    angles_to_visualize[4] = 1
    angles_to_visualize[7] = 1
    angles_to_visualize[8] = 1
    angles_to_visualize[10] = 1
    angles_to_visualize[12] = 1
    angles_to_visualize[14] = 1
    angles_to_visualize[15] = 1

    # Establishing connection with the video
    cap = cv2.VideoCapture(video_path)

    # Properties
    # extracting video properties
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    size = (width, height)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fourcc = cv2.VideoWriter_fourcc('a','v','c','1')
    op_path = os.path.join('data', 'output_videos', 'output.avi')
    # Video Writer 
    video_writer = cv2.VideoWriter(op_path, fourcc, fps, size)

    ## Setup mediapipe instance
    with mp_pose.Pose(min_detection_confidence=0.3, min_tracking_confidence=0.3) as pose:
        while cap.isOpened():
            # Reading frames from live feed
            success, frame = cap.read()

            if not success:
                print("Ignoring empty camera frame.")
                break
            
            # Recolor image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
        
            # Make detection
            results = pose.process(image)
        
            # Recolor back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            
            # Render detections
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                    mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                    )

            # Extract landmarks
            try:
                landmarks = results.pose_landmarks.landmark
            except:
                pass
            
            # we initialize to size (37,3) instead of (33,3) because we include 4 more key-points, for angle calculation
            frame_coordinates = np.zeros((37,3))

            ## extract (x,y,z) coordinate of each landmark
            for i in range(33):
                x, y, z = landmarks[i].x, landmarks[i].y, landmarks[i].z
                frame_coordinates[i][0], frame_coordinates[i][1], frame_coordinates[i][2] = x, y, z
            
            ## creating 4 new landmarks
            # left hand key-point
            frame_coordinates[33] = (frame_coordinates[18] + frame_coordinates[20])/2
            # right hand key-point
            frame_coordinates[35] = (frame_coordinates[17] + frame_coordinates[19])/2
            # neck key-point
            frame_coordinates[34] = (frame_coordinates[11] + frame_coordinates[12])/2
            # middle pelvis key-point
            frame_coordinates[36] = (frame_coordinates[23] + frame_coordinates[24])/2

            # initialize array to hold angles per frame
            angle = np.zeros(k)

            ## extract angles based on the relevant key-points from the keypoint_index
            for i in range(k):
                # calculate the angle based on the key-point indices as in the tuple from keypoint_index
                angle[i] = calculate_angle(frame_coordinates[keypoint_index[i][0]], frame_coordinates[keypoint_index[i][1]], frame_coordinates[keypoint_index[i][2]])

            # Visualize angles
            for i in range(k):
                if angles_to_visualize[i]:
                    theta = int(angle[i] * (180/np.pi))
                    landmark_index = keypoint_index[i][1]
                    # (x,y) coordinates of landmark
                    landmark_coordinates = frame_coordinates[landmark_index][:2]

                    cv2.putText(image,
                                str(theta),
                                tuple(np.multiply(landmark_coordinates, list(size)).astype(int)),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
            
            # write the frame
            video_writer.write(image)



        # Releasing the video capture device
        cap.release()
        # Releasing video writer
        video_writer.release()

# 1. Explanation

- First, we extract the coordinates of 33 keypoints using Mediapipe Blazepose. These are the keypoints and their corresponding labels:
    - ![](https://google.github.io/mediapipe/images/mobile/pose_tracking_full_body_landmarks.png)

- We add 4 more key-points namely:
    - `33`: left-hand
    - `34`: neck
    - `35`: right-hand
    - `36`: middle-pelvis

- We employ cube-normalization by applying the following transformation to each coordinate in each frame:
    - $(x,y,z) \mapsto (\frac{x - x_{\text{min}}}{x_{\text{max}} - x_{\text{min}}}, \frac{y - y_{\text{min}}}{y_{\text{max}} - y_{\text{min}}}, \frac{z - z_{\text{min}}}{z_{\text{max}} - z_{\text{min}}})$
    - This makes $(x_{\text{min}}, y_{\text{min}}, z_{\text{min}})$ the origin of the unit cube coordinate system.

- We then consider 18-relevant joint angles, using the key-point coordinates:
    - ![](https://i.postimg.cc/zBSdvndp/image.png)

- We extract these 18 angles for each frame, and compile these into a `(n, 18)` shaped array, where `n` is the no. of frames in the video.

- Then we use DTW to compare these arrays of the videos to compare and obtain a score

# 2. Code

In [24]:
## helper function: calculating the joint angles
def calculate_angle(a, b, c):
    # we will be calculating angle ABC
    
    # extract unit vector BA:
    v1 = a - b
    v1_u = v1/np.linalg.norm(v1)
    
    # extract unit vector BC:
    v2 = c - b
    v2_u = v2/np.linalg.norm(v2)

    return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))

## helper function: to extract (n, k) dimensional array to store the coordinates of each landmark in each frame

## these are k tuples of keypoints to extract k relevant angles
## k = keypoint_index.shape[0]
keypoint_index = np.array([
    [33, 16, 14],
    [16, 14, 12],
    [14, 12, 11],
    [14, 12, 24],
    [ 0, 34, 12],
    [ 0, 34, 36],
    [12, 11, 13],
    [13, 11, 23],
    [11, 13, 15],
    [13, 15, 35],
    [12, 24, 26],
    [26, 24, 23],
    [24, 23, 25],
    [11, 23, 25],
    [24, 26, 28],
    [23, 25, 27],
    [26, 28, 32],
    [25, 27, 31]
])

def extract_angle_arr(input_video_path, keypoint_index = keypoint_index):
    # no. of angles to consider: k
    k = keypoint_index.shape[0]
    # the list to store the 'k' angles per frame
    angles = []
    
    # extablishing video capture (not necessary if you have extracted the coordinates already in some array, then directly use those info: this is preferable when we are using a limited no. of videos for testing purposes)
    cap = cv2.VideoCapture(input_video_path)

    # setting up mediapipe blazepose
    with mp_pose.Pose(min_detection_confidence=0.3, min_tracking_confidence=0.3) as pose:
        while cap.isOpened():
            # Reading frames from video
            success, frame = cap.read()
            
            if not success:
                print("Ignoring empty camera frame.")
                break
            
            # Recolor image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
        
            # Make detection
            results = pose.process(image)
        
            # Recolor back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            
            # Extract landmarks
            try:
                landmarks = results.pose_landmarks.landmark
            except:
                pass
            
            # we initialize to size (37,3) instead of (33,3) because we include 4 more key-points, for angle calculation
            frame_coordinates = np.zeros((37,3))

            # inialize min-max coordinates variables
            x_min, y_min, z_min = np.inf, np.inf, np.inf
            x_max, y_max, z_max = -np.inf, -np.inf, -np.inf

            ## extract (x,y,z) coordinate of each landmark
            for i in range(33):
                x, y, z = landmarks[i].x, landmarks[i].y, landmarks[i].z
                frame_coordinates[i][0], frame_coordinates[i][1], frame_coordinates[i][2] = x, y, z

                # checking and updating the min coordinates
                if x < x_min:
                    x_min = x
                if y < y_min:
                    y_min = y
                if z < z_min:
                    z_min = z
                
                # checking and updating the max coordinates
                if x > x_max:
                    x_max = x
                if y > y_max:
                    y_max = y
                if z > z_max:
                    z_max = z
            
            ## creating 4 new landmarks
            # left hand key-point
            frame_coordinates[33] = (frame_coordinates[18] + frame_coordinates[20])/2
            # right hand key-point
            frame_coordinates[35] = (frame_coordinates[17] + frame_coordinates[19])/2
            # neck key-point
            frame_coordinates[34] = (frame_coordinates[11] + frame_coordinates[12])/2
            # middle pelvis key-point
            frame_coordinates[36] = (frame_coordinates[23] + frame_coordinates[24])/2

            ## normalize frame_coordinates into a unit cube with (x_min, y_min, z_min) as the origin
            for i in range(37):
                # normalizing x coordinate
                frame_coordinates[i][0] = (frame_coordinates[i][0] - x_min)/(x_max - x_min)
                # normalizing y coordinate
                frame_coordinates[i][1] = (frame_coordinates[i][1] - y_min)/(y_max - y_min)
                # normalizing z coordinate
                frame_coordinates[i][2] = (frame_coordinates[i][2] - z_min)/(z_max - z_min)

            # initialize array to hold angles per frame
            angle = np.zeros(k)

            ## extract angles based on the relevant key-points from the keypoint_index
            for i in range(k):
                # calculate the angle based on the key-point indices as in the tuple from keypoint_index
                angle[i] = calculate_angle(frame_coordinates[keypoint_index[i][0]], frame_coordinates[keypoint_index[i][1]], frame_coordinates[keypoint_index[i][2]])
            
            angles.append(angle)

    # Releasing the video capture
    cap.release()

    # convert the list into an np array of size: (n, k)
    return np.array(angles)




from dtaidistance.dtw_ndim import distance, distance_fast

## to compare 2 videos of dim (n, k) using DTW
def compare_vid(vid1_arr, vid2_arr, senstivity = 1):
    ## normalize the coordinates using sensitivity; higher sensitivity => more lenient scoring, lower sensitivity => strict scoring

    vid1_arr = vid1_arr/(np.linalg.norm(vid1_arr) * senstivity)
    vid2_arr = vid2_arr/(np.linalg.norm(vid2_arr) * senstivity)
    
    # calculate distance
    d = distance(vid1_arr, vid2_arr)
    # we give a score out of 100
    d_score = 100 - (d*100)
    
    return d_score



## comparing videos by path: just including the pre-processing steps in the previous function
def combined_compare(vid1_path, vid2_path, senstivity=1):
    vid1_arr = extract_angle_arr(vid1_path)
    vid2_arr  = extract_angle_arr(vid2_path)

    score = compare_vid(vid1_arr, vid2_arr, senstivity)

    print("Score:", score)
    return score

# 3. Testing

In [25]:
## helper function to compare right videos with every other right video and every wrong video to right video and give average right and average wrong scores

def compare_right_wrong_vid(right_paths, wrong_paths):
    # comparing right videos with each other
    r_score = 0
    r_count = 0

    for i in range(len(right_paths)):
        for j in range(i+1, len(right_paths)):
            r_score += combined_compare(right_paths[i], right_paths[j])
            r_count += 1
    

    # comparing wrong videos with right videos
    w_score = 0
    w_count = 0

    for i in range(len(right_paths)):
        for j in range(len(wrong_paths)):
            w_score += combined_compare(right_paths[i], wrong_paths[j])
            w_count += 1
    

    # print out the average scores
    print("Average Right Video Comparison Score:", r_score/r_count)
    print("Average Wrong Video Comparison Score:", w_score/w_count)

In [26]:
# extract video paths

# this is the format to follow to read the video paths
'''
r1_path = os.path.join('data', 'non-vanilla_test_data', 'exercise_name', 'R1.1.mp4')
r2_path = os.path.join('data', 'non-vanilla_test_data', 'exercise_name', 'R1.2.mp4')
w1_path = os.path.join('data', 'non-vanilla_test_data', 'exercise_name', 'W2.1.mp4')
w2_path = os.path.join('data', 'non-vanilla_test_data', 'exercise_name', 'W2.2.mp4')
'''

"\nr1_path = os.path.join('data', 'exercise_name', 'R1.1.mp4')\nr2_path = os.path.join('data', 'exercise_name', 'R1.2.mp4')\nw1_path = os.path.join('data', 'exercise_name', 'W2.1.mp4')\nw2_path = os.path.join('data', 'exercise_name', 'W2.2.mp4')\n"

## 1. Cable Bar Curls

In [31]:
# extract video paths

r1_path = os.path.join('data', 'non-vanilla_test_data', 'cableBarCurls', 'R1.1.mp4')
r2_path = os.path.join('data', 'non-vanilla_test_data', 'cableBarCurls', 'R1.2.mp4')
w1_path = os.path.join('data', 'non-vanilla_test_data', 'cableBarCurls', 'W2.1.mp4')
w2_path = os.path.join('data', 'non-vanilla_test_data', 'cableBarCurls', 'W2.2.mp4')

r_paths = [r1_path, r2_path]
w_paths = [w1_path, w2_path]

In [33]:
compare_right_wrong_vid(r_paths, w_paths)

Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 89.14991920043461
Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 46.979874979781314
Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 61.09276857550815
Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 53.06287335139201
Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 66.85947754075687
Average Right Video Comparison Score: 89.14991920043461
Average Wrong Video Comparison Score: 56.99874861185958


In [34]:
# extract a visualization
create_visualization(r1_path)

Ignoring empty camera frame.


## 2. Deadlift

In [35]:
# extract video paths

r1_path = os.path.join('data', 'non-vanilla_test_data', 'deadlift', 'R1.1.mp4')
r2_path = os.path.join('data', 'non-vanilla_test_data', 'deadlift', 'R1.2.mp4')
w1_path = os.path.join('data', 'non-vanilla_test_data', 'deadlift', 'W2.1.mp4')
w2_path = os.path.join('data', 'non-vanilla_test_data', 'deadlift', 'W2.2.mp4')

r_paths = [r1_path, r2_path]
w_paths = [w1_path, w2_path]

In [36]:
compare_right_wrong_vid(r_paths, w_paths)

Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 92.12561189490877
Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 67.6504185386642
Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 71.13574131573282
Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 68.44258993337988
Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 71.45158376885081
Average Right Video Comparison Score: 92.12561189490877
Average Wrong Video Comparison Score: 69.67008338915693


In [37]:
# extract a visualization
create_visualization(r1_path)

Ignoring empty camera frame.


## 3. Straight Leg Kickbacks

In [49]:
# extract video paths

r1_path = os.path.join('data', 'non-vanilla_test_data', 'straightLegKickBacks', 'R1.1.mp4')
r2_path = os.path.join('data', 'non-vanilla_test_data', 'straightLegKickBacks', 'R1.2.mp4')
w1_path = os.path.join('data', 'non-vanilla_test_data', 'straightLegKickBacks', 'W2.1.mp4')
w2_path = os.path.join('data', 'non-vanilla_test_data', 'straightLegKickBacks', 'W2.2.mp4')

r_paths = [r1_path, r2_path]
w_paths = [w1_path, w2_path]

In [50]:
compare_right_wrong_vid(r_paths, w_paths)

Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 91.4533835262837
Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 69.75813874067939
Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 68.18027765552351
Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 76.93731415901529
Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 75.54205155919934
Average Right Video Comparison Score: 91.4533835262837
Average Wrong Video Comparison Score: 72.60444552860437


In [51]:
# extract a visualization
create_visualization(r1_path)

Ignoring empty camera frame.


## 4. Reverse Fly

In [43]:
# extract video paths

r1_path = os.path.join('data', 'non-vanilla_test_data', 'reverseFly', 'R1.1.mp4')
r2_path = os.path.join('data', 'non-vanilla_test_data', 'reverseFly', 'R1.2.mp4')
w1_path = os.path.join('data', 'non-vanilla_test_data', 'reverseFly', 'W2.1.mp4')
w2_path = os.path.join('data', 'non-vanilla_test_data', 'reverseFly', 'W2.2.mp4')

r_paths = [r1_path, r2_path]
w_paths = [w1_path, w2_path]

In [44]:
compare_right_wrong_vid(r_paths, w_paths)

Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 94.37287377117931
Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 85.5332480964526
Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 81.00508756289773
Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 87.71333930212397
Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 84.18538364044745
Average Right Video Comparison Score: 94.37287377117931
Average Wrong Video Comparison Score: 84.60926465048044


In [45]:
# extract a visualization
create_visualization(r1_path)

Ignoring empty camera frame.


## 5. Plank

In [46]:
# extract video paths

r1_path = os.path.join('data', 'non-vanilla_test_data', 'plank', 'R1.1.mp4')
r2_path = os.path.join('data', 'non-vanilla_test_data', 'plank', 'R1.2.mp4')
w1_path = os.path.join('data', 'non-vanilla_test_data', 'plank', 'W2.1.mp4')
w2_path = os.path.join('data', 'non-vanilla_test_data', 'plank', 'W2.2.mp4')

r_paths = [r1_path, r2_path]
w_paths = [w1_path, w2_path]

In [47]:
compare_right_wrong_vid(r_paths, w_paths)

Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 89.07789432536592
Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 82.6742562197015
Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 69.19085114933112
Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 84.74352407546873
Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 68.35083031146648
Average Right Video Comparison Score: 89.07789432536592
Average Wrong Video Comparison Score: 76.23986543899196


In [48]:
# extract a visualization
create_visualization(r1_path)

Ignoring empty camera frame.


## 6. Lunges

In [52]:
# extract video paths

r1_path = os.path.join('data', 'non-vanilla_test_data', 'lunges', 'R1.1.mp4')
r2_path = os.path.join('data', 'non-vanilla_test_data', 'lunges', 'R1.2.mp4')
w1_path = os.path.join('data', 'non-vanilla_test_data', 'lunges', 'W2.1.mp4')
w2_path = os.path.join('data', 'non-vanilla_test_data', 'lunges', 'W2.2.mp4')

r_paths = [r1_path, r2_path]
w_paths = [w1_path, w2_path]

In [53]:
compare_right_wrong_vid(r_paths, w_paths)

Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 85.67162065829733
Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 82.82896995031554
Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 82.14782193063787
Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 76.30588164671367
Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 76.22446589623385
Average Right Video Comparison Score: 85.67162065829733
Average Wrong Video Comparison Score: 79.37678485597525


In [54]:
# extract a visualization
create_visualization(r1_path)

Ignoring empty camera frame.


## 7. Front Arm Raises

In [55]:
# extract video paths

r1_path = os.path.join('data', 'non-vanilla_test_data', 'frontArmRaise', 'R1.1.mp4')
r2_path = os.path.join('data', 'non-vanilla_test_data', 'frontArmRaise', 'R1.2.mp4')
w1_path = os.path.join('data', 'non-vanilla_test_data', 'frontArmRaise', 'W2.1.mp4')
w2_path = os.path.join('data', 'non-vanilla_test_data', 'frontArmRaise', 'W2.2.mp4')

r_paths = [r1_path, r2_path]
w_paths = [w1_path, w2_path]

In [56]:
compare_right_wrong_vid(r_paths, w_paths)

Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 93.05305289295296
Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 66.71510872328682
Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 71.7991619894982
Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 73.49533747370944
Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 78.40156645002233
Average Right Video Comparison Score: 93.05305289295296
Average Wrong Video Comparison Score: 72.60279365912919


In [57]:
# extract a visualization
create_visualization(r1_path)

Ignoring empty camera frame.


## 8. Donkey Kick

In [66]:
# extract video paths

r1_path = os.path.join('data', 'non-vanilla_test_data', 'donkeyKick', 'R1.1.mp4')
r2_path = os.path.join('data', 'non-vanilla_test_data', 'donkeyKick', 'R1.2.mp4')
w1_path = os.path.join('data', 'non-vanilla_test_data', 'donkeyKick', 'W2.1.mp4')
w2_path = os.path.join('data', 'non-vanilla_test_data', 'donkeyKick', 'W2.2.mp4')

r_paths = [r1_path, r2_path]
w_paths = [w1_path, w2_path]

In [59]:
compare_right_wrong_vid(r_paths, w_paths)

Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 85.30634907282868
Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 72.29168594998531
Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 85.98184515014701
Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 84.88709312258841
Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 85.98423018019112
Average Right Video Comparison Score: 85.30634907282868
Average Wrong Video Comparison Score: 82.28621360072796


In [67]:
# extract a visualization
create_visualization(r1_path)

Ignoring empty camera frame.


## 9. Bulgarian Split Squats

In [71]:
# extract video paths

r1_path = os.path.join('data', 'non-vanilla_test_data', 'bulgarianSplitSquats', 'R1.1.mp4')
r2_path = os.path.join('data', 'non-vanilla_test_data', 'bulgarianSplitSquats', 'R1.2.mp4')
w1_path = os.path.join('data', 'non-vanilla_test_data', 'bulgarianSplitSquats', 'W2.1.mp4')
w2_path = os.path.join('data', 'non-vanilla_test_data', 'bulgarianSplitSquats', 'W2.2.mp4')

r_paths = [r1_path, r2_path]
w_paths = [w1_path, w2_path]

In [72]:
compare_right_wrong_vid(r_paths, w_paths)

Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 91.87764475790397
Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 84.3347686018169
Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 85.43503430466023
Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 81.83641783216899
Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 82.61782778215877
Average Right Video Comparison Score: 91.87764475790397
Average Wrong Video Comparison Score: 83.55601213020122


In [73]:
# extract a visualization
create_visualization(r1_path)

Ignoring empty camera frame.


## 10. Boat Pose

In [81]:
# extract video paths

r1_path = os.path.join('data', 'non-vanilla_test_data', 'boatPose', 'R1.1.mp4')
r2_path = os.path.join('data', 'non-vanilla_test_data', 'boatPose', 'R1.2.mp4')
w1_path = os.path.join('data', 'non-vanilla_test_data', 'boatPose', 'W2.1.mp4')
w2_path = os.path.join('data', 'non-vanilla_test_data', 'boatPose', 'W2.2.mp4')

r_paths = [r1_path, r2_path]
w_paths = [w1_path, w2_path]

In [82]:
compare_right_wrong_vid(r_paths, w_paths)

Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 94.92409455573463
Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 80.69083770589035
Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 80.37257321250205
Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 80.97555271615909
Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 80.5909439770787
Average Right Video Comparison Score: 94.92409455573463
Average Wrong Video Comparison Score: 80.65747690290755


In [83]:
# extract a visualization
create_visualization(r1_path)

Ignoring empty camera frame.


## 11. Dumbbell Curls

In [84]:
# extract video paths

r1_path = os.path.join('data', 'non-vanilla_test_data', 'dumbbellCurls', 'R1.1.mp4')
r2_path = os.path.join('data', 'non-vanilla_test_data', 'dumbbellCurls', 'R1.2.mp4')
r3_path = os.path.join('data', 'non-vanilla_test_data', 'dumbbellCurls', 'R1.3.mp4')
w1_path = os.path.join('data', 'non-vanilla_test_data', 'dumbbellCurls', 'W2.1.mp4')
w2_path = os.path.join('data', 'non-vanilla_test_data', 'dumbbellCurls', 'W2.2.mp4')

r_paths = [r1_path, r2_path, r3_path]
w_paths = [w1_path, w2_path]

In [85]:
compare_right_wrong_vid(r_paths, w_paths)

Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 86.9791943786166
Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 82.48480494259894
Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 81.87120014194673
Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 80.57485928147821
Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 72.04266675764134
Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 80.58970989682184
Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 76.03272737020221
Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 83.4072183973026
Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 76.91644868978454
Average Right Video Comparison Score: 83.7783998210541
Average Wrong Video Comparison Score: 78.26060506553846


In [77]:
# extract a visualization
create_visualization(r1_path)

Ignoring empty camera frame.


In [ ]:
%%time